## Implementation of Huber and Pseudo-Huber loss for regression and its Variant for classification

<table class="colab-notebook-buttons" align="center" width="50%";>
  <td><center>
    <a target="_blank" href="https://colab.research.google.com/github/Atharva-Gundawar/Huber_and_friends/blob/main/Huber_and_friends.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    </br>Run in Google Colab</a>
  </center></td>
  <td><center>
    <a target="_blank" href="https://github.com/Atharva-Gundawar/Huber_and_friends/blob/main/Huber_and_friends.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    </br>View source on GitHub</a>
  </center></td>
</table>

## Preparing the colab 

In [4]:
import tensorflow as tf
import numpy as np


In [3]:
X = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
Y = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

In [7]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units = 1, input_shape=[1])])
model.compile(optimizer='sgd', loss ='mean_squared_error')
model.fit(X,Y,epochs=500,verbose=0)
print(model.predict([10.0]))

[[18.981476]]


## Here we will implement the Huber loss function
Huber Loss :
</br><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/0f9f57a0796cdc6476d4c4093b74864884b08c40" alt="MarineGEO circle logo" style="height: 100px; width:100px;"/>

In [5]:
def return_huber_loss(threshold = 1):
    @tf.function
    def huber_loss(y_true,y_pred):
        error = y_true - y_pred
        is_small = tf.abs(error) < threshold
        small_error = tf.square(error)/2
        big_error = threshold * (tf.abs(error) - (0.5 * threshold))
        return tf.where(is_small,small_error,big_error)
    return huber_loss

In [6]:
huber_loss = return_huber_loss()

In [37]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units = 1, input_shape=[1])])
model.compile(optimizer='sgd', loss =huber_loss)
model.fit(X,Y,epochs=500,verbose=0)
print(model.predict([10.0]))

[[18.751842]]


## Here we will implement the Pseudo Huber loss function
Pseudo Huber Loss :
</br><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/ddb74a0b410409d421e880aa65f57c20393f91c1" alt="MarineGEO circle logo" style="height: 100px; width:100px;"/>

In [2]:
def return_pseudo_huber_loss(threshold = 1):
    @tf.function
    def pseudo_huber_loss(y_true,y_pred):
        threshold = tf.convert_to_tensor(threshold, dtype=tf.float32)
        error = y_true - y_pred
        underoot = tf.sqrt(1 + tf.square(threshold/error))
        return threshold * (underoot - 1)
    return pseudo_huber_loss

In [7]:
pseudo_huber_loss = return_pseudo_huber_loss()

In [41]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units = 1, input_shape=[1])])
model.compile(optimizer='sgd', loss =pseudo_huber_loss)
model.fit(X,Y,epochs=500,verbose=0)
print(model.predict([10.0]))

[[-1.6253761]]


## Here we will implement the Variant for classification function
Variant for classification :
</br><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/c2efccbf2c3001686259291146ca484c75d48fac" alt="MarineGEO circle logo" style="height: 100px; width:100px;"/>

In [9]:
def return_huber_loss_for_classification(threshold=1):
    @tf.function
    def huber_loss_for_classification(fx,y):
        """
        For classification purposes, a variant of the Huber loss called modified Huber is sometimes used. 
        Given a prediction f(x) (a real-valued classifier score) and a true binary class label y.
        """
        differentiator = fx * y
        is_small = differentiator >= -1
        small_error = tf.square(tf.maximum(0,differentiator))
        big_error = -4*differentiator
        return tf.where(is_small,small_error,big_error)
    return huber_loss_for_classification